In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import pickle
from statistics import mean 

### Load data

In [2]:
NgeNetEx = pd.read_csv('D:/mesh2mesh/finalResults/results_exvivo_ngenet.csv')
RegTREx = pd.read_csv('D:/mesh2mesh/finalResults/results_exvivo_regtr.csv')

NgeNetIn = pd.read_csv('D:/mesh2mesh/finalResults/results_invivo_ngenet2.csv')
RegTRIn = pd.read_csv('D:/mesh2mesh/finalResults/results_invivo_regtr2.csv')

NgeNetExVar = pd.read_csv('D:/mesh2mesh/finalResults/results_exvivo_ngenet_variance.csv')
RegTRExVar = pd.read_csv('D:/mesh2mesh/finalResults/results_exvivo_regtr_variance.csv')

## Dataset

In [4]:
# Create an overlay histogram with the two datasets
fig = go.Figure()
fig.add_trace(go.Histogram(x=NgeNetExVar.overlap_scores, name='High variance dataset', marker_color='#1f77b4', opacity=0.75, nbinsx=50))
fig.add_trace(go.Histogram(x=NgeNetEx.overlap_scores, name='Normal dataset', marker_color='#ff7f0e', opacity=0.75, nbinsx=10))
fig.update_layout(barmode='overlay', xaxis_title='Visible point ratio [%]', yaxis_title='Count', width=800, height=400)
fig.show()
fig.write_image('DataGen/visible_point_ratio_histogram_overlay.pdf')

In [82]:
with open('D:/mesh2mesh/mesh_dataset/ear_dataset/volume_distribution.pkl', 'rb') as f:
    vd_syn = pickle.load(f)

with open('D:/mesh2mesh/mesh_dataset/DIOME_FanShapeCorr/volume_distribution.pkl', 'rb') as f:
    vd_real = pickle.load(f)

tympanic_membrane_syn = []
tympanic_membrane_real = []
malleus_syn = []
malleus_real = []
incus_syn = []
incus_real = []
stapes_syn = []
stapes_real = []
for i in vd_syn:
    tympanic_membrane_syn.append(i['tympanic_membrane'])
    malleus_syn.append(i['malleus'])
    incus_syn.append(i['incus'])
    stapes_syn.append(i['stapes'])

for i in vd_real:
    tympanic_membrane_real.append(i['tympanic_membrane'])
    malleus_real.append(i['malleus'])
    incus_real.append(i['incus'])
    stapes_real.append(i['stapes'])

d = {
    'Dataset': ['Ex vivo dataset', 'Ex vivo dataset','Ex vivo dataset','Ex vivo dataset', 'In vivo dataset', 'In vivo dataset', 'In vivo dataset', 'In vivo dataset'],
    'Segment': ['Tympanic membrane', 'Malleus', 'Incus', 'Stapes', 'Tympanic membrane', 'Malleus', 'Incus', 'Stapes'],
    'Volume ratio': [mean(tympanic_membrane_syn), mean(malleus_syn), mean(incus_syn), mean(stapes_syn), mean(tympanic_membrane_real), mean(malleus_real), mean(incus_real), mean(stapes_real)]
}

df = pd.DataFrame(data=d)
fig = px.bar(df, x='Dataset', y="Volume ratio", color="Segment", log_y=False, width=800, height=400)
fig.show()
fig.write_image('DataGen/volume_ratio_barplot.pdf')

In [39]:
vd_syn

[{'tympanic_membrane': 0.7758462946020128,
  'malleus': 0.15599268069533395,
  'incus': 0.042543458371454713,
  'stapes': 0.025617566331198535},
 {'tympanic_membrane': 0.8214116532449076,
  'malleus': 0.12079583135954523,
  'incus': 0.04216011369019422,
  'stapes': 0.015632401705352912},
 {'tympanic_membrane': 0.7881429816913688,
  'malleus': 0.14646904969485613,
  'incus': 0.03923278116826504,
  'stapes': 0.026155187445510025},
 {'tympanic_membrane': 0.8053658536585366,
  'malleus': 0.12536585365853659,
  'incus': 0.04926829268292683,
  'stapes': 0.02},
 {'tympanic_membrane': 0.7780362959516054,
  'malleus': 0.15821312238250348,
  'incus': 0.041879944160074456,
  'stapes': 0.021870637505816658},
 {'tympanic_membrane': 0.8234482758620689,
  'malleus': 0.1310344827586207,
  'incus': 0.022068965517241378,
  'stapes': 0.023448275862068966},
 {'tympanic_membrane': 0.7991285403050109,
  'malleus': 0.13769063180827887,
  'incus': 0.03790849673202614,
  'stapes': 0.025272331154684097},
 {'tym

## C2P-Net

### Ex vivo

In [6]:
x = [round(i*2)/2 for i in RegTREx.nonregistered_cd_l]


fig = go.Figure()

fig.add_trace(go.Box(
    y=[i for u, i in enumerate(RegTREx.mean_displacement_error_l) if x[u] != 0],
    x=[round(i*2)/2 for i in RegTREx.nonregistered_cd_l if round(i*2)/2 != 0],
    name='C2P-Net [RegTR]',
    marker_color='#3D9970'
))
fig.add_trace(go.Box(
    y=[i for u, i in enumerate(NgeNetEx.mean_displacement_error_l) if x[u] != 0],
    x=[round(i*2)/2 for i in RegTREx.nonregistered_cd_l if round(i*2)/2 != 0],
    name='C2P-Net [NgeNet]',
    marker_color='#FF4136'
))

fig.update_layout(
    xaxis_title='Initial error (CD) [mm]',
    yaxis_title='Mean Displacement Error [mm]',
    boxmode='group', # group together boxes of the different traces for each value of x
    width=1.5*800, height=1.5*400
)
fig.show()
fig.write_image('C2P-Net/ex-vivo/regtr_vs_ngenet_vs_initial_registration_error.pdf')

In [28]:
x = [round(i*2)/2 for i in RegTREx.nonregistered_cd_l]

fig = go.Figure()

fig.add_trace(go.Box(
    y=RegTREx.mean_displacement_error_l,
    x=[round(i*0.01)/0.01 for i in RegTREx.len_inds],
    name='C2P-Net [RegTR]',
    marker_color='#3D9970'
))
fig.add_trace(go.Box(
    y=NgeNetEx.mean_displacement_error_l,
    x=[round(i*0.01)/0.01 for i in NgeNetEx.len_coors_l],
    name='C2P-Net [NgeNet]',
    marker_color='#FF4136'
))

fig.update_layout(
    xaxis_title='Number of corresponding pairs',
    yaxis_title='Mean Displacement Error [mm]',
    boxmode='group', # group together boxes of the different traces for each value of x
    width=1.5*800, height=1.5*400
)
fig.show()
fig.write_image('C2P-Net/ex-vivo/regtr_vs_ngenet_vs_number_of_correspondences.pdf')

In [80]:
x = [round(i*2)/2 for i in RegTREx.nonregistered_cd_l]

fig = go.Figure()

fig.add_trace(go.Box(
    y=RegTREx.mean_displacement_error_l,
    x=[round(i*2)/2 for i in RegTREx.regtr_cd_l],
    name='C2P-Net [RegTR]',
    marker_color='#43c2c2'
))
fig.add_trace(go.Box(
    y=NgeNetEx.mean_displacement_error_l,
    x=[round(i*2)/2 for i in NgeNetEx.ngenet_cd_l],
    name='C2P-Net [NgeNet]',
    marker_color='#f1832f'
))

fig.update_layout(
    xaxis_title='Rigid stage registration error (CD) [mm]',
    yaxis_title='Mean Displacement Error [mm]',
    boxmode='group', # group together boxes of the different traces for each value of x
    width=1.5*800, height=1.5*400
)
fig.show()
fig.write_image('C2P-Net/ex-vivo/regtr_vs_ngenet_vs_rigid_stage_registration_error.pdf')

In [5]:
from math import ceil
x = [round(i*10)/10 for i in NgeNetExVar.overlap_scores]

fig = go.Figure()

fig.add_trace(go.Box(
    y=RegTRExVar.mean_displacement_error_l,
    x=[ceil(i*10)/10 for i in RegTRExVar.overlap_scores],
    name='C2P-Net [RegTR]',
    marker_color='#3395e3'
))
fig.add_trace(go.Box(
    y=NgeNetExVar.mean_displacement_error_l,
    x=[ceil(i*10)/10 for i in NgeNetExVar.overlap_scores],
    name='C2P-Net [NgeNet]',
    marker_color='#45b372'
))

fig.update_layout(
    xaxis_title='Visible point ratio [%]',
    yaxis_title='Mean Displacement Error [mm]',
    boxmode='group', # group together boxes of the different traces for each value of x
    width=1.1*800, height=1.1*400
)
#fig.update_yaxes(type="log")
fig.show()
fig.write_image('C2P-Net/ex-vivo/regtr_vs_ngenet_vs_visible_point_ratio_high_variance.pdf')

### In vivo

Comparison NgeNet vs RegTR vs non registered LANDMARK LOSS

In [9]:
num = 31
samples = [f'Sample {i}' for i in range(num) if RegTRIn.non_registered_landmark_loss_l[i] != -1]

fig = go.Figure(data=[
    go.Bar(name='Unregistered', x=samples, y=RegTRIn.non_registered_landmark_loss_l[:num]),
    go.Bar(name='C2P-Net [NgeNet]', x=samples, y=NgeNetIn.landmark_loss_l[:num]),
    go.Bar(name='C2P-Net [RegTR]', x=samples, y=RegTRIn.landmark_loss_l[:num])
])
# Change the bar mode
fig.update_layout(yaxis_title='Landmark Loss [mm]', xaxis_title= 'All Samples of DIOME', barmode='group', width=1000, height=500)
fig.show()
fig.write_image('C2P-Net/in-vivo/ngenet_vs_regtr_vs_unregistered-bargraph31.pdf')

In [4]:
import plotly.graph_objects as go

x = ['Left 42%' if i == 'left' else 'Right 58%' for i in NgeNetIn.side]

fig = go.Figure()

fig.add_trace(go.Box(
    y=[i if i != -1 else float('nan') for i in RegTRIn.nonregistered_cd_l],
    x=x,
    name='Unregistered',
    #marker_color='#3D9970',
    boxpoints=False,
))

fig.add_trace(go.Box(
    y=[i if i != -1 else float('nan') for i in NgeNetIn.landmark_loss_l],
    x=x,
    name='C2P-Net [NgeNet]',
    #marker_color='#3D9970',
    boxpoints=False,
))

fig.add_trace(go.Box(
    y=RegTRIn.landmark_loss_l,
    x=x,
    name='C2P-Net [RegTR]',
    #marker_color='#FF4136',
    boxpoints=False,
))

fig.update_layout(
    yaxis_title='Landmark Error [mm]',
    boxmode='group',
    width=1000, height=500,
)
fig.show()
fig.write_image('C2P-Net/in-vivo/right_vs_left_corrected_landmark_loss.pdf')

In [5]:
import plotly.graph_objects as go

x = ['Healthy 67%' if i == 'healthy' else 'Diseased 33%' for i in NgeNetIn.status]

fig = go.Figure()

fig.add_trace(go.Box(
    y=[i if i != -1 else float('nan') for i in RegTRIn.nonregistered_cd_l],
    x=x,
    name='Unregistered',
    #marker_color='#3D9970',
    boxpoints=False,
))

fig.add_trace(go.Box(
    y=[i if i != -1 else float('nan') for i in NgeNetIn.landmark_loss_l],
    x=x,
    name='C2P-Net [NgeNet]',
    #marker_color='#3D9970',
    boxpoints=False,
))
fig.add_trace(go.Box(
    y=RegTRIn.landmark_loss_l,
    x=x,
    name='C2P-Net [RegTR]',
    #marker_color='#FF4136',
    boxpoints=False,
))

fig.update_layout(
    yaxis_title='Landmark Error [mm]',
    boxmode='group',
    width=1000, height=500,
    
)
fig.show()
fig.write_image('C2P-Net/in-vivo/healthy_vs_unhealthy_landmark_loss.pdf')

## NgeNet

### Ex vivo

In [12]:
import plotly.express as px
fig = px.scatter(x=NgeNetEx.ngenet_cd_l, y=NgeNetEx.mean_displacement_error_l, trendline="lowess", trendline_color_override="red", width=800, height=400)
fig.show()

In [13]:
import plotly.express as px
fig = px.scatter(x=NgeNetExVar.overlap_scores, y=NgeNetExVar.mean_displacement_error_l, trendline="ols", trendline_color_override="red", width=1*800, height=1*400)
fig.update_layout(
    xaxis_title="visible point ratio [%]", yaxis_title="Mean Displacement Error [mm]"
)
fig.show()

In [15]:
import plotly.express as px
df = px.data.tips()
fig = px.box(x=[round(i*10)/10 for i in NgeNetExVar.overlap_scores], y=NgeNetExVar.mean_displacement_error_l, width=1*800, height=1*400, log_y=True)
fig.update_layout(
    xaxis_title="Visible point ratio [%]", yaxis_title="Mean Displacement Error [mm]"
)
fig.show()

In [20]:
import plotly.express as px
df = px.data.tips()
fig = px.box(x=[round(i*0.01)/0.01 for i in NgeNetEx.len_coors_l], y=NgeNetEx.mean_displacement_error_l, width=1*800, height=1*400)
fig.update_layout(
    xaxis_title="Number of discovered correspondences", yaxis_title="Mean Displacement Error [mm]"
)
fig.show()
fig.write_image('NgeNet/ex-vivo/num_correspondeces_vs_mde.pdf')

### In vivo

## RegTR

### Ex vivo

In [24]:
fig = px.box(x=[round(i*2)/2 for i in RegTREx.regtr_cd_l], y=RegTREx.mean_displacement_error_l, width=1*800, height=1*400)
fig.update_layout(
    xaxis_title="RegTR CD score [mm]", yaxis_title="Mean Displacement Error [mm]"
)
fig.show()

In [26]:
import plotly.express as px
df = px.data.tips()
fig = px.box(x=[round(i*0.01)/0.01 for i in RegTREx.len_inds], y=RegTREx.mean_displacement_error_l, width=1*800, height=1*400)
fig.update_layout(
    xaxis_title="Number of discovered correspondences", yaxis_title="Mean Displacement Error [mm]"
)
fig.show()
fig.write_image('RegTR/ex-vivo/num_correspondeces_vs_mde.pdf')

In [ ]:
import plotly.express as px
from math import floor, round

fig = px.box(x=[round(i*50)/50 for i in pred_overlap_score.mean(axis=1)], y=mean_displacement_error_l, width=1*800, height=1*400, log_y=True)
fig.update_layout(
    xaxis_title="mean overlap confidence", yaxis_title="Mean Displacement Error"
)
fig.show()
fig.write_image('d2ear_IPCAI2023_images/REGTR_mean_overlap_confidence_to_mde.pdf')

In [ ]:
import plotly.express as px
from math import floor, round

fig = px.box(x=[round(i*2)/2 for i in registered_partial_cd_l], y=mean_displacement_error_l, width=1*800, height=1*400)
fig.update_layout(
    xaxis_title="Partial CD", yaxis_title="Mean Displacement Error"
)
fig.show()
#fig.write_image('d2ear_IPCAI2023_images/REGTR_partialCD_vs_mde_boxplot.png')

### In vivo

In [9]:
RegTRIn.status

0            healthy
1            healthy
2            healthy
3            healthy
4      cholesteatoma
5            healthy
6            healthy
7            healthy
8            healthy
9            healthy
10           healthy
11     cholesteatoma
12           healthy
13    reconstruction
14           healthy
15           healthy
16         sclerosis
17           healthy
18           healthy
19           healthy
20      otitis media
21       perforation
22    reconstruction
23           healthy
24        retraction
25    reconstruction
26        retraction
27           healthy
28           healthy
29           healthy
30           healthy
31       perforation
32     cholesteatoma
33           healthy
34           healthy
35    reconstruction
36    reconstruction
37           healthy
38           healthy
39           healthy
40           healthy
41           healthy
42           healthy
Name: status, dtype: object